## Setup

In [8]:
# import statements
import xarray as xr
import numpy as np
import pandas as pd

In [56]:
# function because some of the simulations have a messed-up calendar
def preprocess(ds_in: xr.Dataset) -> xr.Dataset:
    ds_in.XTIME.attrs["units"] = "minutes since 2021-09-01 23:00:00"
    ds_in.XTIME.attrs["description"] = "minutes since 2021-09-01 23:00:00"
    return(ds_in)

def load(var: str, init_time: str, ens_num: int):
    path = "/mnt/drive2/wof-runs/"+init_time+"/"+var+"/wrfwof*.{:02d}".format(ens_num)
    return(xr.open_mfdataset(path,preprocess=preprocess,
                          decode_times=False,combine="nested",
                          concat_dim="Time")
    )
def fix_xtime(ds_in: xr.Dataset,init_time: str) -> xr.Dataset:
    if init_time in ["20", "21", "22", "23"]:
        start_date = "2021-09-01"
    elif init_time in ["00", "01", "02"]:
        start_date = "2021-09-02"
    else:
        raise ValueError("Unexpected init_time provided.")
    ds_in.XTIME.attrs["units"] = "minutes since "+start_date+" "+init_time+":00:00"
    ds_in.XTIME.attrs["description"] = ("minutes since "+start_date+" "+
                                        init_time+":00:00")
    return(xr.decode_cf(ds_in))

## Read in WoFS simulations

In [57]:
rainnc_20Z = xr.concat([fix_xtime(load("RAINNC","20Z",e),"20") for e in range(1,19)],
                       "ens")
rainnc_22Z = xr.concat([fix_xtime(load("RAINNC","22Z",e),"22") for e in range(1,19)],
                       "ens")

## Find the point closest to Central Park

In [63]:
# Set desired latitude and longitude
nyc_lat,nyc_lon = 40.7826,-73.9656

# Isolate XLAT and XLONG for ease of use
xlat = rainnc_20Z.XLAT.isel(Time=0).squeeze().drop_vars('XTIME')
xlong = rainnc_20Z.XLONG.isel(Time=0).squeeze().drop_vars('XTIME')

# Calculate the Euclidean distance between XLAT and XLONG values and the desired point
distance = np.sqrt((xlat-nyc_lat)**2 + (xlong-nyc_lon)**2)

# Find where the distance is minimized
nyc_inds = distance.argmin(dim=["south_north","west_east"])

In [64]:
type(nyc_inds['south_north'])

xarray.core.dataarray.DataArray

## Calculate 1-hour accumulated rainfall for 1-2Z

In [69]:
def get_NYC_1Z_to_2Z(ds_in: xr.Dataset, lat_index: xr.DataArray,
                      lon_index: xr.DataArray):
    hour = ds_in.isel(Time=abs(rainnc_20Z.XTIME - np.datetime64(
        pd.to_datetime("2021-09-02 02:00:00"))).argmin()) - ds_in.isel(Time=abs(
            rainnc_20Z.XTIME - np.datetime64(
                pd.to_datetime("2021-09-02 01:00:00"))).argmin())
    return(hour.isel(south_north = lat_index,west_east = lon_index))

In [71]:
rainy_hour_20Z = get_NYC_1Z_to_2Z(
    rainnc_20Z,nyc_inds["south_north"],nyc_inds["west_east"]
    ).RAINNC

rainy_hour_22Z = get_NYC_1Z_to_2Z(
    rainnc_22Z,nyc_inds["south_north"],nyc_inds["west_east"]
    ).RAINNC